In [ ]:
from app.factor.factorModel import factorModel
import copy
import numpy as np
from tqdm import tqdm
m = factorModel()





m.factor_name_lst = ['Analyst_factor', 'NegMktValue']
Equity_Idx_Monthly_Equity_Returns, Monthly_Equity_Returns, Monthly_Factor_Score, Equity_Idx_Monthly_Factor_Score, Daily_Equity_Returns, benchmark_dailyret = m.getData()
lookBackPeriod = 6
stock_names = list(Equity_Idx_Monthly_Factor_Score.keys())

month_score_copy = copy.deepcopy(Monthly_Factor_Score)
equity_idx_month_score_copy = copy.deepcopy(Equity_Idx_Monthly_Factor_Score)

for idx, factor in tqdm(enumerate(m.factor_name_lst)):
    print(f'-----FACTOR: {factor}------')

    for i in tqdm(range(lookBackPeriod, len(m.bt_tradedate))):
        period = m.bt_tradedate[i-lookBackPeriod:i]
        endDate = m.pre_bt_tradedate[i]
        month = m.bt_tradedate[i]

        _, _, _, _, df_bt_alpha_indicator = m.calculate(Equity_Idx_Monthly_Equity_Returns, Monthly_Equity_Returns, month_score_copy, equity_idx_month_score_copy, Daily_Equity_Returns, benchmark_dailyret,
                                                                        month_names = period,
                                                                        main = False,
                                                                        endTimeAdj = endDate,
                                                                        factor_idx=[idx],
                                                                        factor_names = [factor])

        df_bt_alpha_indicator = df_bt_alpha_indicator[:-1]
        bestGroup = df_bt_alpha_indicator['calmar'].idxmax()

        if bestGroup == 0:
            continue

        #上面我们看了1-6月的数据 找到了最好的组数量
        #下面要看7月份的分组，去看组x里的股票是什么，然后把7月权重改了
        nameList, scoreList = [], []
        for name in stock_names:
            name, score = m.calcEquityScore(name, [1], equity_idx_month_score_copy, month, [idx])
            if name:
                nameList.append(name)
                scoreList.append(score)
        equityGroups = m.rankEquity(nameList, scoreList)

        stocksToAdjust = equityGroups[bestGroup] #要调整的股票名字

        #getting the stocks that are avail this month
        currMonthkeys = [k for k, v in equity_idx_month_score_copy.items() if month in v]
        
        #getting the keys of those stocks to adjust of all stocks
        keyOfStocks = [currMonthkeys.index(i) for i in stocksToAdjust]
        
        #adjust Monthly_Factor_Score of all of those stocks to be larger
        currMonthFactorScore = Monthly_Factor_Score[factor][month]
        for i in keyOfStocks:
            currMonthFactorScore[i] += max(currMonthFactorScore)
        currMonthFactorScore = (currMonthFactorScore - np.mean(currMonthFactorScore)) / np.std(currMonthFactorScore)
        Monthly_Factor_Score[factor][month] = currMonthFactorScore

        #adjust Equity_Idx_Monthly_Factor_Score
        for stock, stockidx in zip(stocksToAdjust, keyOfStocks):
            Equity_Idx_Monthly_Factor_Score[stock][month][idx] = Monthly_Factor_Score[factor][month][stockidx]

In [ ]:
from app.factor.factorModel import factorModel
import itertools
import numpy as np
from tqdm import tqdm
m = factorModel()

combToIC = {}

m.factor_name_lst = m.allfactorname_lst
_, Monthly_Equity_Returns, Monthly_Factor_Score, _, _, _ = m.getData()
print('gotData')
factors = m.factor_name_lst
months = list(Monthly_Equity_Returns.keys())

for factorComb in tqdm(itertools.combinations(factors, 7)):
    currIC = 0
    for month in months:
        returns = Monthly_Equity_Returns[month]
        for factor in factorComb:
            if not scores:
                scores = Monthly_Factor_Score[factor][month]
            else:
                scores = np.add(scores, Monthly_Factor_Score[factor][month])
        corr = np.corrcoef(scores, returns)[0,1]
        currIC += corr
        print(f'for month {month}, comb {factorComb} has IC {corr}')
    combToIC[factorComb] = currIC
    print(f'Comb {factorComb} has IC {combToIC[factorComb]}')


    




In [ ]:
Monthly_Factor_Score['gmmDmean1m20D']['20210104']

In [1]:
from app.factor.factorModel import factorModel
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import time
m = factorModel()

# res = pd.DataFrame(columns=['F1','F2','F3','F4','F5','F6','F7','IC'])

m.factor_name_lst = m.allfactorname_lst
print("getting data")
currT = time.time()
_, Monthly_Equity_Returns, Monthly_Factor_Score, _, _, _ = m.getData()
print(f'gotData in {time.time() - currT} seconds')
# factors = m.factor_name_lst
# months = list(Monthly_Equity_Returns.keys())

# chooseAmount = 7

df = pd.DataFrame()

for factor, factor_data in Monthly_Factor_Score.items():
    data = []
    month_list = []
    for month, scores in factor_data.items():
        month_list.extend([month]*len(scores))
        data.extend(scores)
    df_factor = pd.DataFrame(data, columns=[factor], index=month_list)
    df = pd.concat([df, df_factor], axis=1)

returns = []
month_list_ret = []
for month, ret in Monthly_Equity_Returns.items():
    month_list_ret.extend([month]*len(ret))
    returns.extend(ret)

df_returns = pd.DataFrame(returns, columns=['Returns'], index=month_list_ret)
df = pd.concat([df, df_returns], axis=1)

print(df)

df.to_csv('factorScoreToReturn.csv')


getting data
gotData in 113.62610101699829 seconds
          Analyst_factor  NegMktValue  technology_factor   tps_sps  \
20210104        0.396948    -6.376436          -0.458922  0.244044   
20210104        0.184236    -5.973033          -0.608177  0.249567   
20210104       -0.237436     0.427529           0.750740  0.215019   
20210104       -0.261377     0.314248          -0.138511  0.171653   
20210104       -0.031587     0.332653           0.495872  0.249693   
...                  ...          ...                ...       ...   
20230801       -0.538152     0.277543           0.200400  0.113201   
20230801        0.061928     0.370996          -0.426301 -0.334429   
20230801        0.412090     0.389597           0.288977 -0.258804   
20230801        0.091448    -2.282325          -0.934665  0.159859   
20230801        0.236228     0.028438          -0.413215  0.181306   

          momentumn_factor  avgwght_momentum   seven_f  udslDWL  udslUCL  \
20210104          0.761350      

In [ ]:
def worker(factorComb, res, lock):
    currIC = 0
    for month in months:
        scores = np.array([])
        returns = Monthly_Equity_Returns[month]
        for factor in factorComb:
            if scores.size == 0:
                scores = np.array(Monthly_Factor_Score[factor][month])
            else:
                scores = np.add(scores, Monthly_Factor_Score[factor][month])
        corr = np.corrcoef(scores, returns)[0,1]
        currIC += corr
    tempArr = list(factorComb) + [currIC]
    lock.acquire()
    try:
        res.loc[len(res)] = tempArr
    finally:
        lock.release()

pool = mp.Pool(processes=mp.cpu_count())  # Create a pool of processes
lock = mp.Manager().Lock() 

for factorComb in itertools.combinations(factors, chooseAmount):
    pool.apply_async(worker, args=(factorComb, res, lock))

print('finished Mapping')

pool.close()
pool.join()

res.to_csv(f'factorChoose{chooseAmount}.csv')

In [ ]:
df_bt_alpha_indicator

In [ ]:

lookBackPeriod = 6

for i in range(lookBackPeriod, len(m.bt_tradedate)):
    print('period is ', m.bt_tradedate[i-lookBackPeriod:i])
    print('endDate is ', m.pre_bt_tradedate[i])
    print('for calc of the trading mo of ', m.bt_tradedate[i])

In [ ]:
from app.factor.factorModel import factorModel
m = factorModel()
m.start = '20230101'
m.end = '20230731'
m.factor_name_lst = ['NegMktValue']
Equity_Idx_Monthly_Equity_Returns, Monthly_Equity_Returns, Monthly_Factor_Score, Equity_Idx_Monthly_Factor_Score, Daily_Equity_Returns, benchmark_dailyret = m.getData()

_, _, _, _, df_bt_alpha_indicator = m.calculate(Equity_Idx_Monthly_Equity_Returns, Monthly_Equity_Returns, Monthly_Factor_Score, Equity_Idx_Monthly_Factor_Score, Daily_Equity_Returns, benchmark_dailyret,
                                                                month_names = ['20230201', '20230301', '20230403', '20230504', '20230601', '20230703'],
                                                                main = True,
                                                                endTimeAdj = '20230731')

df_bt_alpha_indicator = df_bt_alpha_indicator[:-1]
bestGroup = df_bt_alpha_indicator['calmar'].idxmax()
print(df_bt_alpha_indicator)
print(bestGroup)
print(list(Monthly_Equity_Returns.keys()))

In [ ]:
len(m.pre_bt_tradedate)

In [ ]:
len(m.bt_tradedate)

In [ ]:
m = factorModel()
m.start = '20221001'
m.end = '20230731'
m.bt_tradedate

In [ ]:
lookBack = 6

for i in range(lookBack, len(m.bt_tradedate)):
    monthName = m.bt_tradedate[i-lookBack:i]
    print(monthName)

In [ ]:
minMonth = 2
maxMonth = 5

availMonth = len(month_names)

if availMonth < maxMonth:
    startMonth = month_names[0]
else:
    startMonth = month_names[availMonth-maxMonth]

res = defaultdict(list)

endMonth = month_names[-1]

for factor in Monthly_Factor_Score:
    for date in Monthly_Factor_Score[factor]:
        #print(date, int(date) >= int(startMonth), int(date) <= int(endMonth))
        if int(date) >= int(startMonth) and int(date) <= int(endMonth):
            res[factor] += Monthly_Factor_Score[factor][date]

res = pd.DataFrame(res)

ICList = ICList.tail(min(maxMonth, ICList.shape[0]))

covar = np.cov(ICList, rowvar=False)
corr = np.corrcoef(res, rowvar=False)
D = np.diag(np.sqrt(np.diag(covar)))
covar = D @ nlg.inv(corr) @ D

mat = nlg.inv(covar)                 
weight = mat*np.mat(ICList.mean()).reshape(len(mat),1)
weight = np.array(weight.reshape(len(weight),))[0]
weight.tolist()






In [ ]:
import seaborn as sns
sns.heatmap(ICList.corr(), annot=True)

In [ ]:
import seaborn as sns
sns.heatmap(res.corr(), annot=True)

In [ ]:
ic

In [ ]:
np.corrcoef(ICList, rowvar=False).shape

In [ ]:
#corr to covar
covar = np.cov(ICList)
corr = np.corrcoef(res)

D = np.diag(np.sqrt(np.diag(covar)))
covar = D @ corr @ D

In [ ]:
# mat = nlg.inv(covar)                    
# weight = mat*np.mat(IC.mean()).reshape(len(mat),1)

# weight = np.array(weight.reshape(len(weight),))[0]
# weight = weight.tolist()

In [ ]:
corr = np.corrcoef(ICList)
covar = np.cov(ICList)

In [ ]:
newScore = defaultdict(list)
for k, v in Monthly_Factor_Score.items():
    newScore[key] = 
    # newLst[key] = np.vstack([np.array(i) for i in Monthly_Factor_Score[key].values()])



In [ ]:
res.keys()

In [ ]:
Monthly_Factor_Score['decay_panic'].values()

In [ ]:
#covar to corr
Dinv = np.diag(1 / np.sqrt(np.diag(covar))) 
corr = Dinv @ covar @ Dinv

In [ ]:
#corr to covar
D = np.diag(np.sqrt(np.diag(covar)))
covar = D @ corr @ D

In [ ]:
from app.factor.factorModel import factorModel
f = factorModel()
a, b, c, d, e, f = f.getData()

In [ ]:
c

In [ ]:
for stk in d:
    for val in d[stk]:
        if len(d[stk][val]) != 13:
            print('not equal to 13')

In [ ]:
d

In [ ]:
import pandas as pd

df = pd.read_csv('factor.csv',header=None)

print(df[0].tolist())